In [1]:
import pandas as pd
from pymystem3 import Mystem
import numpy as np
from gensim.models.phrases import Phraser, Phrases
from gensim.models import TfidfModel, Word2Vec, FastText
from gensim import corpora
import nltk
import matplotlib.pyplot as plt
from collections import namedtuple, Counter
import re
import glob
import os
import matplotlib.pyplot as plt

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.graph_objs import *

import artm

%matplotlib inline

print(artm.version())

0.9.0


In [2]:
df = pd.read_csv('labirint.csv', error_bad_lines=False)

In [3]:
df.reset_index(inplace=True)

In [6]:
#df.drop('index', axis=1, inplace=True)
df.columns = ['0','title', 'content']
df=df[300000:500000]

In [9]:
df.index = range(len(df))

In [3]:
%%time
#df = pd.read_csv('test.csv')
df.fillna('', inplace=True)
df['comment'] = df.title + ' ' + df.content


drop_comment = []
alph = 'йцукенгшщзхъфывапролджэячсмитьбю'
for i in range(len(df)):
    delete = False
    for char in df.comment.loc[i]:
        if char in alph:
            delete = False
            break
        else:
            delete = True
    if delete:
        drop_comment.append(i)
df.drop(drop_comment,axis=0,inplace=True)
df.index = range(len(df))


df['lemma'] = ''

m = Mystem()

idx = 0
for comment in df.comment.as_matrix():
    lemma = m.lemmatize(comment)[:]
    df.lemma.loc[idx] = ''.join(lemma[1:len(lemma)- 1])
    if idx%20000 == 0:
        print(idx)
    idx +=1

0
CPU times: user 833 ms, sys: 66.3 ms, total: 899 ms
Wall time: 3.53 s


In [4]:
%%time
drop_comment = []
alph = 'йцукенгшщзхъфывапролджэячсмитьбю'
for i in range(len(df)):
    delete = False
    for char in df.lemma.loc[i]:
        if char in alph:
            delete = False
            break
        else:
            delete = True
    if delete:
        drop_comment.append(i)
df.drop(drop_comment,axis=0,inplace=True)
df.index = range(len(df))

CPU times: user 60.6 ms, sys: 0 ns, total: 60.6 ms
Wall time: 59.9 ms


In [5]:
stop_word = ["в",'!','?' "без", "до", "из", "к", "на", "по", "о", "от", "перед",
             "при", "через", "за", "над", "об", "под", "про", "для", "вблизи", 
             "вглубь", "вдоль", "возле", "около", "вокруг", "впереди", "после", 
             "посредством", "в","роли", "путём", "насчёт", "по","поводу", "ввиду", "по", "случаю", 
             "в", "течение", "благодаря", "несмотря", "на", "спустя", "с", "из-под", 
             "из-за", "я", "быть", "с", "это", "все", "то", "по-над", "отличие", "от", 
             "в","связи", "как", "словно", "так","как","т.к.", "тоже", "зато", "чтобы", 
             "также", "потому","что", "и", "а", "что", "или", "но", "однако", "когда", "лишь", 
             "едва", "где", "куда", "откуда", "столько", "настолько", "так", "такой","степени", "до",
             "того", "такой",  "будто", "точно", "если",  "коли", "ежели", "несмотря", "хотя", "хоть", 
             "пускай", "дабы", "-","же",":" "ли", "какой", "этот", "весь", "бы", "то", "у", 
             "уже","который","там","оно","ничто","кто","вот","тот","ну","со" "даже","она","он", ' ']

In [6]:
%%time
from time import time as t

#df = pd.read_csv('ml_tags1.csv')
#df = df[['id','client','ext','store','author','lang','country','tag','content']]

t0 = t()
wpt = nltk.TweetTokenizer()
tokenized_corpus_viz = [wpt.tokenize(re.sub('[/\nt,.qwtyuipadghjklzxcbn$&^=+"]', '', str(document))) for document in df['lemma'].str.lower()]
print('tokenize end')
tmp = []
z = 0
for j in tokenized_corpus_viz:
    d = [doc not in stop_word for doc in j]
    d = list(np.array(j)[d])
    k = 0
    dtemp = []
    for i in range(len(d)):
        #print(d[i])
        if d[i] == 'не':
            try:
                dtemp.append(d[i] + '_' + d[i+1])
                k =1
            except:
                0
        else:
            if k == 0:
                dtemp.append(d[i])
            else:
                k = 0
    d = dtemp
    tmp.append(d)
    if z%100000 ==0:
        print(z)
    z+=1
tokenized_corpus_viz = tmp
t1 = t()
print("%s: %.2g sec" % ('tokenize end: ', int(t1 - t0)))

tokenize end
0
tokenize end: : 0 sec
CPU times: user 139 ms, sys: 218 µs, total: 139 ms
Wall time: 137 ms


In [7]:
%%time
fastText = FastText.load('fast.model')
new_tokenize =[]
k = 0
t0 = t()
for rew in tokenized_corpus_viz:
    new_rew = []
    for word in rew:
        try:
            
            if fastText.wv.vocab[word].index > 1400:
                if fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][1] >= 0.52:
                    new_rew.append(fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][0])
            else:
                new_rew.append(word)
        except:
            try:
                if fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][1] >= 0.52:
                    new_rew.append(fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][0])
            except:
                0
    new_tokenize.append(new_rew)
    if k%100000 == 0:
        print(k)
        print(t() - t0)
        t0 = t()
    k+=1

0
0.3070704936981201
CPU times: user 3.25 s, sys: 2.94 s, total: 6.19 s
Wall time: 3.36 s


In [8]:
%%time
bigram_transformer = Phraser.load('bi_trans.phr')
trigram_trasformer = Phraser.load('tri_trans.phr')

def tex_gen_trigram(tokenized_corpus):
    trig = []
    for text in tokenized_corpus:
        trig.append(trigram_trasformer[bigram_transformer[[word for word in text]]])
    return trig

trigram_corp = tex_gen_trigram(new_tokenize)

CPU times: user 1.68 s, sys: 257 ms, total: 1.93 s
Wall time: 1.79 s


In [9]:
%%time
w2v = Word2Vec.load('w2v.model')
new_tokenize =[]
k = 0
t0 = t()
for rew in trigram_corp:
    new_rew = []
    for word in rew:
        try:
            if w2v.wv.vocab[word].index > 1700:
                new_rew.append(model.wv.most_similar(w2v, restrict_vocab=1700, topn=1)[0][0])
            else:
                new_rew.append(word)
        except:
            0
    new_tokenize.append(new_rew)
    if k%100000 == 0:
        print(k)
        print(t() - t0)
        t0 = t()
    k+=1

0
0.00010704994201660156
CPU times: user 195 ms, sys: 16.3 ms, total: 211 ms
Wall time: 210 ms


/home/but/.local/lib/python3.5/site-packages/gensim/utils.py:491: DeprecationWarning:

Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).



In [10]:
new_stop_words = ['(',')','программа','приложение','только','много','еще','просто', 'прога','виджет','очень','нет','они','очень','можно','ни','»','«','—','ли','пока','очень','просто','вообще','пк','со','сам','чем','oo','тут','потом','даже','якобы','другой','они','два','много','либо','ты']

In [11]:
%%time
tokenize_corpus = []
for j in new_tokenize:
    d = [doc not in new_stop_words for doc in j]
    d = list(np.array(j)[d])
    tokenize_corpus.append(d)

CPU times: user 24 ms, sys: 2.38 ms, total: 26.3 ms
Wall time: 25.2 ms


In [12]:
%%time
corpus = tokenize_corpus
dictionary = corpora.Dictionary(corpus)
corpus = [dictionary.doc2bow(line) for line in corpus]

CPU times: user 46 ms, sys: 0 ns, total: 46 ms
Wall time: 45.5 ms


In [13]:
%%time
k = 0
with open("reviews.vw", "wb") as outfile:
    for j in corpus:
        k += 1
        line = 'doc' + str(k) + ' '
        for i in j:
            if dictionary[i[0]] != ':':
                line += dictionary[i[0]] + ':' + str(i[1]) + ' '
        line += "\n"
        outfile.write(line.encode('utf-8'))

CPU times: user 34.3 ms, sys: 4.33 ms, total: 38.7 ms
Wall time: 39.9 ms


In [16]:
%%time
dictionary = artm.Dictionary()

batch_vectorizer = None
if len(glob.glob(os.path.join('lab', '*.batch'))) < 1:
    batch_vectorizer = artm.BatchVectorizer(data_path='reviews.vw', data_format='vowpal_wabbit', collection_name='lab', target_folder='lab')
else:
    batch_vectorizer = artm.BatchVectorizer(data_path='kos2', data_format='batches')
    
model_artm = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(65)], num_processors=8, num_document_passes=45,
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    
                                                    dictionary=dictionary)],
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)],
                       cache_theta=True)
if not os.path.isfile('kos/dictionary.dict'):
    dictionary.gather(data_path=batch_vectorizer.data_path)
    dictionary.save(dictionary_path='kos/dictionary.dict')

dictionary.load(dictionary_path='kos/dictionary.dict')
model_artm.initialize(dictionary=dictionary)

model_artm.load('good_artm1.1')

CPU times: user 118 ms, sys: 7.45 ms, total: 126 ms
Wall time: 137 ms


In [17]:
%%time
a = model_artm.transform(batch_vectorizer=batch_vectorizer) #, theta_matrix_type='dense_ptdw'
b = model_artm.transform(batch_vectorizer=batch_vectorizer, theta_matrix_type='dense_ptdw')

CPU times: user 216 ms, sys: 24.4 ms, total: 240 ms
Wall time: 236 ms


'topic_0',  
'topic_1 - позитив',  
'topic_2 - плохое соединение/тормозит',  
'topic_3 - банк',  
'topic_4 - период',  
'topic_5 - позитив',  
'topic_6 - негатив/спам',  
'topic_7 - ',  
'topic_8 - позитив',  
'topic_9 - тех поддержка',  
'topic_10 - баг',  
'topic_11 - карты',  
'topic_12 - товар',  
'topic_13 - позитив',   
'topic_14 - предложение',  
'topic_15 - пароль/вход',  
'topic_16 - позитив / полезное',  
'topic_17 - позитив / удобно',  
'topic_18 - инфа',  
'topic_19 - баг/предложение',  
'topic_20 - больше фич',  
'topic_21 - больше возможностей',  
'topic_22 - баг',  
'topic_23 - позитив / обращение к разрабам',  
'topic_24 - проблемы входа/регистрации',   
'topic_25 - долго',  
'topic_26 - баг',  
'topic_27 - деньги/оплата',   
'topic_28',  
'topic_29',  
'topic_30 - позитив',  
'topic_31 - язык/позитив',  
'topic_32 - позитив',  
'topic_33 - ',  
'topic_34 - оплата',  
'topic_35 - тупит/глючит/нормально',  
'topic_36 - верните',  
'topic_37 - дизайн',  
'topic_38 - позитив',  
'topic_39 - позитив',  
'topic_40 - место',  
'topic_41 - позитив',   
'topic_42 - вопрос',  
'topic_43 - позитив👍',  
'topic_44 - баг/не устанавливается',  
'topic_45 - фунционал',  
'topic_46 - обращение/предложение/благодарность',  
'topic_47 - читать',  
'topic_48 - сервис',  
'topic_49 - баг / жрет батарею',  
'topic_50 - езда/маршрут',  
'topic_51 - контакт/звонок',  
'topic_52 - интерфейс',  
'topic_53 - не грузится/не отправляется',  
'topic_54 - позитив(?)',  
'topic_55 - баг / вылетает / не заходит',   
'topic_56 - сложно',  
'topic_57 - промо',  
'topic_58 - позитив/норм',   
'topic_59 - позитив/выручает'  

In [20]:
topic = ['topic_0',
'topic_1 - позитив',
'topic_2 - плохое соединение/тормозит',
'topic_3 - банк',
'topic_4 - период',
'topic_5 - позитив',
'topic_6 - негатив/спам',
'topic_7 - ',
'topic_8 - позитив',
'topic_9 - тех поддержка',
'topic_10 - баг',
'topic_11 - карты',
'topic_12 - товар',
'topic_13 - позитив',
'topic_14 - предложение',
'topic_15 - пароль/вход',
'topic_16 - позитив / полезное',
'topic_17 - позитив / удобно',
'topic_18 - инфа',
'topic_19 - баг/предложение',
'topic_20 - больше фич',
'topic_21 - больше возможностей',
'topic_22 - баг',
'topic_23 - позитив / обращение к разрабам',
'topic_24 - проблемы входа/регистрации',
'topic_25 - долго',
'topic_26 - баг',
'topic_27 - деньги/оплата',
'topic_28',
'topic_29',
'topic_30 - позитив',
'topic_31 - язык/позитив',
'topic_32 - позитив',
'topic_33 - ',
'topic_34 - оплата',
'topic_35 - тупит/глючит/нормально',
'topic_36 - верните',
'topic_37 - дизайн',
'topic_38 - позитив',
'topic_39 - позитив',
'topic_40 - место',
'topic_41 - позитив',
'topic_42 - вопрос',
'topic_43 - позитив👍',
'topic_44 - баг/не устанавливается',
'topic_45 - фунционал',
'topic_46 - обращение/предложение/благодарность',
'topic_47 - читать',
'topic_48 - сервис',
'topic_49 - баг / жрет батарею',
'topic_50 - езда/маршрут',
'topic_51 - контакт/звонок',
'topic_52 - интерфейс',
'topic_53 - не грузится/не отправляется',
'topic_54 - позитив(?)',
'topic_55 - баг / вылетает / не заходит',
'topic_56 - сложно',
'topic_57 - промо',
'topic_58 - позитив/норм',
'topic_59 - позитив/выручает']

In [33]:
import plotly.plotly as py
import plotly.graph_objs as go

pipe = a.sum(axis=1)/a.sum(axis=1).sum()

labels = pipe[pipe>0.02].index
values = pipe[pipe>0.02].values

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~texpomru13/0 or inside your plot.ly account where it is named 'basic_pie_chart'


In [35]:

for top in pipe[pipe>0.02].index:
    for i in a:
        if a[i].loc[top] > 0.3:
            print(df.content.loc[i])
            print(a[i][a[i] != 0])
    print(top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!')

Очень удобное приложение, приемлемые цены, срок доставки кратчайшие. Всем рекомендую
topic_1 - позитив    0.503188
topic_7 -            0.496812
Name: 8, dtype: float32
Очень удобное ! Все работает отлично !
topic_1 - позитив    1.0
Name: 30, dtype: float32
Спасибо! Очень удобно и понятно.
topic_1 - позитив              0.422429
topic_17 - позитив / удобно    0.577571
Name: 34, dtype: float32
Красивое и при этом очень удобное приложение, но самое главное, что магазин очень хороший!
topic_1 - позитив    0.812362
topic_12 - товар     0.187638
Name: 39, dtype: float32
Все очень удобно, удобная навигация. Легко модно найти любую книгу. Спасибо разработчикам!!
topic_1 - позитив              0.380518
topic_11 - карты               0.112422
topic_17 - позитив / удобно    0.318051
topic_47 - читать              0.189010
Name: 42, dtype: float32
Очень удобное приложение! Цены дешевле чем в магазинах! Спасибо!
topic_1 - позитив    0.531123
topic_7 -            0.219799
topic_12 - товар     0.249

Name: 423, dtype: float32
Люблю лабиринт, приложение очень удобное в использовании! Лойс👍
topic_1 - позитив     0.562958
topic_13 - позитив    0.437042
Name: 431, dtype: float32
Очень удобное приложение, отслеживаются акции и скидки.
topic_1 - позитив    0.499327
topic_18 - инфа      0.500673
Name: 438, dtype: float32
Приложение очень понравилось! Так удобно! Спасибо разработчикам!
topic_1 - позитив              0.685184
topic_17 - позитив / удобно    0.314816
Name: 439, dtype: float32
В общем приложение отличное . Радует)
topic_1 - позитив    1.0
Name: 460, dtype: float32
Хорошее приложение, где быстро можно найти необходимое, даже полностью не 
вводя название.
topic_1 - позитив                        0.333229
topic_17 - позитив / удобно              0.350025
topic_24 - проблемы входа/регистрации    0.316746
Name: 466, dtype: float32
Очень удобное приложение
topic_1 - позитив    1.0
Name: 486, dtype: float32
Приложение отличное, легко найти любую книгу! Спасибо разработчикам и 
админи

topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!topic_7 - !!!!!!!!!!!!!
Отличный интернет магазин
topic_2 - плохое соединение/тормозит    0.459526
topic_12 - товар                        0.540474
Name: 6, dtype: float32
Удобно, часто есть скидки.
topic_12 - товар               0.609294
topic_17 - позитив / удобно    0.390706
Name: 13, dtype: float32
Отличный ассортимент, стоимость товаров, скорость исполнения заказа, приятные курьеры. 
НО, крайне неудобный поиск в приложении, при введение названия товара ( не конкретную книгу конкретного автора, а расходные товары, например скотч) в строку поиска, предлагаются максимум 3 неподходящих товара, все приходится искать через меню, что очень длительно!
topic_2 - плохое соединение/тормозит     0.044126
topic_12 - товар                         0.476747
topic_22 - баг                      

Самый лучший интернет-магазин какой я встречала
topic_5 - позитив    0.615361
topic_12 - товар     0.384639
Name: 811, dtype: float32
Один из лучших магазинов!!!
topic_12 - товар    1.0
Name: 817, dtype: float32
Это лучший книжный интернет магазин
topic_2 - плохое соединение/тормозит    0.373627
topic_12 - товар                        0.626373
Name: 820, dtype: float32
Очень удобное приложение и отличный магазин!!! Радует своими скидками!
topic_1 - позитив    0.402157
topic_12 - товар     0.597843
Name: 825, dtype: float32
Очень нравится приложение, вот только с субботы перестало работать. И очень 
долго обновляется персональная скидка. Уже несколько дней после покупки 
прошло, а она не изменилась.
topic_12 - товар                     0.350071
topic_25 - долго                     0.136287
topic_30 - позитив                   0.097681
topic_35 - тупит/глючит/нормально    0.193434
topic_56 - сложно                    0.222527
Name: 829, dtype: float32
topic_12 - товар!!!!!!!!!!!!!topic_1

Name: 327, dtype: float32
Просто красавчики!! Понравилось информативность и точность доставки.
topic_2 - плохое соединение/тормозит    0.211360
topic_17 - позитив / удобно             0.588971
topic_48 - сервис                       0.199669
Name: 330, dtype: float32
Хорошее приложение, очень удобно, спасибо.
topic_17 - позитив / удобно    1.0
Name: 332, dtype: float32
Отличное приложение!
topic_17 - позитив / удобно    1.0
Name: 336, dtype: float32
Быстро, доступно, качественно и без проблем.
topic_17 - позитив / удобно    1.0
Name: 339, dtype: float32
Удобно, быстро, часто скидки и акции. Главное, намного дешевле, чем в магазине!
topic_12 - товар               0.2874
topic_17 - позитив / удобно    0.7126
Name: 341, dtype: float32
Никогда не висло, не вылетало. Всё очень удобно! Спасибо!
topic_17 - позитив / удобно    1.0
Name: 343, dtype: float32
Спасибо за отличное приложение! Удобно, быстро, понятно.
topic_17 - позитив / удобно    1.0
Name: 346, dtype: float32
Удобно. Спасибо.
topi

Name: 807, dtype: float32
topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!topic_17 - позитив / удобно!!!!!!!!!!!!!
Ребята, всё хорошо, но адаптируйте уже для iphone x, ведь не мало времени прошло.
Upd: Спасибо. Желаю вам удачи и дальнейшего развития.
topic_23 - позитив / обращение к разрабам    0.691704
topic_56 - сложно                            0.308296
Name: 48, dtype: float32
Молодцы и продукты и разработчики.
topic_12 - товар                             0.277383
topic_23 - позитив / обращение к разрабам    0.722617
Name: 53, dtype: float32
Очень удобное приложение! Все сделано по уму. Плюс скидки и выгодные приложения. Спасибо!)
topic_1 - позитив                            

Name: 101, dtype: float32
Все книги заказываю в лабиринте и через приложение!
topic_47 - читать    0.5
topic_48 - сервис    0.5
Name: 105, dtype: float32
Хайли рекомменд!отличнейшее приложение для покупки любимых и новых книг!все удобно,подробно и понятно!
topic_12 - товар               0.225457
topic_17 - позитив / удобно    0.237097
topic_47 - читать              0.537446
Name: 124, dtype: float32
Вылетает сразу же, чуть было не купил книгу в Читай-городе, исправьте!
topic_35 - тупит/глючит/нормально    0.429883
topic_47 - читать                    0.315694
topic_50 - езда/маршрут              0.254423
Name: 135, dtype: float32
Такое ощущение будто бы здесь есть все книги. Очень радует как само приложение, довольно быстро работающее, так и сам метод оплаты практически в одно касание
topic_17 - позитив / удобно    0.428200
topic_45 - фунционал           0.075251
topic_47 - читать              0.496549
Name: 171, dtype: float32
Приложение хорошее, акции хорошие, не все книги правда быв

Сервис оставляет желать лучшего
topic_48 - сервис    1.0
Name: 265, dtype: float32
Мое любимое приложение для заказа книг
topic_47 - читать    0.685185
topic_48 - сервис    0.314815
Name: 289, dtype: float32
Замечательное приложение, удобное. Быстрая доставка, вежливые курьеры. Очень понравилось колесо )
topic_1 - позитив     0.495935
topic_30 - позитив    0.186320
topic_48 - сервис     0.317745
Name: 291, dtype: float32
Очень приятно удивили!!!👏👏👏👏👏👏 спасибо! И так заказываю много книг , сейчас буду заказывать ещё больше!!)💋
topic_13 - позитив    0.396241
topic_37 - дизайн     0.104010
topic_47 - читать     0.172916
topic_48 - сервис     0.326834
Name: 299, dtype: float32
Удобное приложение! Всегда заказываю книги только через него
topic_47 - читать    0.632019
topic_48 - сервис    0.367982
Name: 326, dtype: float32
Лабиринт из лав 💚💚💚 спасибо за отличный сервис.
topic_48 - сервис    1.0
Name: 344, dtype: float32
Очень удобный сервис)
topic_1 - позитив    0.5
topic_48 - сервис    0.5


topic_17 - позитив / удобно                       0.334434
topic_46 - обращение/предложение/благодарность    0.284223
topic_52 - интерфейс                              0.381342
Name: 464, dtype: float32
Делаю первый заказ, оформление супербыстрое, в эту пятницу можно будет 
получить. Ставлю пока 4, дальше будет видно
topic_30 - позитив      0.138362
topic_48 - сервис       0.129052
topic_52 - интерфейс    0.371531
topic_56 - сложно       0.210484
topic_57 - промо        0.150570
Name: 468, dtype: float32
Отлично, только вот новый дизайн чересчур мрачный. Красный, конечно, 
отвлекал своей яркостью, но был хотя-бы не такой траурный
topic_6 - негатив/спам    0.323009
topic_37 - дизайн         0.268249
topic_52 - интерфейс      0.408742
Name: 470, dtype: float32
Привык к красному логотипу этого приложения и когда пришлось переустановить 
приложение, и увидел это черное лого, посчитал по началу что кто-то закосил 
под оригинальный магазин Лабиринта. Капец)))) не сразу решился установить 
да